In [2]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

import numpy as np
import random
import os
import math
import pandas

from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Building an Environment

In [3]:
path = "./data.csv"
number_of_STA = 3


def get_min_snr(state):
    df = pandas.read_csv(path)

    for i in range(1, number_of_STA + 1):
        temp = df.iloc[state][i]

        if i == 1:
            min_snr = temp
            continue

        if min_snr > temp:
            min_snr = temp

    return min_snr


def get_opt_snr(mcs):
    if mcs == 5:
        opt_snr = 27
    else:
        opt_snr = 32

    return opt_snr

In [4]:
mcs = 5

a = 30
b = 1.2
c = 15.02221

d = 75
f = 0.9
g = 10.6280


def get_reward(state):
    opt_snr = get_opt_snr(mcs)

    min_snr = get_min_snr(state)

    if opt_snr < min_snr:
        reward = -a + (math.e - b) ** (min_snr - opt_snr - c)
    else:
        reward = -d + (math.e - f) ** (min_snr - opt_snr + g)

    return reward

In [5]:
class SimulatorEnv(Env):

    def __init__(self):
        self.action_space = Discrete(22)
        self.observation_space = Discrete(22)
        self.state = 21
        self.simulation_time = 10

    def step(self, action):
        self.state = action

        self.simulation_time -= 1

        reward_value = get_reward(self.state)

        if self.simulation_time <= 0:
            done = True
        else:
            done = False

        info = {}

        return self.state, reward_value, done, info

    def render(self):
        pass

    def reset(self):
        self.state = 21
        self.simulation_time = 10

        return self.state
    

# Tests

## Test reward function

In [6]:
for state in range(0, 22):

    print('AP power: {} Reward: {}'.format(state, get_reward(state)))

AP power: 0 Reward: -74.1615988242503
AP power: 1 Reward: -73.59762431271574
AP power: 2 Reward: -72.55006682729756
AP power: 3 Reward: -70.58668707601386
AP power: 4 Reward: -58.076776260648224
AP power: 5 Reward: -41.68396819542581
AP power: 6 Reward: -33.19072937757327
AP power: 7 Reward: 12.630493319444497
AP power: 8 Reward: 57.784002572178025
AP power: 9 Reward: 114.78739101330805
AP power: 10 Reward: 194.98375427533455
AP power: 11 Reward: 308.88414875560466
AP power: 12 Reward: 470.8040590896105
AP power: 13 Reward: -29.997673867356976
AP power: 14 Reward: -29.997025618779997
AP power: 15 Reward: -29.996196715751164
AP power: 16 Reward: -29.995136813337794
AP power: 17 Reward: -29.993781536439535
AP power: 18 Reward: -29.99204856984962
AP power: 19 Reward: -29.989832658691064
AP power: 20 Reward: -29.986999215570368
AP power: 21 Reward: -29.983376146167416


## Test environment

In [7]:
episodes = 5
env = SimulatorEnv()

for episode in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward

    print('Episode:{} Score:{}'.format(episode, score))

env.close()


Episode:1 Score:194.6239620682664
Episode:2 Score:-445.4310198271626
Episode:3 Score:-37.679792489510035
Episode:4 Score:699.5128902072328
Episode:5 Score:35.742006033610025


# Train model

In [8]:
env = SimulatorEnv()

In [9]:
log_path = os.path.join('logs')
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)
model.learn(total_timesteps=100000)    

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to logs\DQN_6
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 226      |
|    exploration_rate | 0.996    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 540      |
|    time_elapsed     | 0        |
|    total_timesteps  | 40       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 105      |
|    exploration_rate | 0.992    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 608      |
|    time_elapsed     | 0        |
|    total_timesteps  | 80       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_r

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 241      |
|    exploration_rate | 0.916    |
| time/               |          |
|    episodes         | 88       |
|    fps              | 630      |
|    time_elapsed     | 1        |
|    total_timesteps  | 880      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 255      |
|    exploration_rate | 0.913    |
| time/               |          |
|    episodes         | 92       |
|    fps              | 634      |
|    time_elapsed     | 1        |
|    total_timesteps  | 920      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 261      |
|    exploration_rate | 0.909    |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 164      |
|    exploration_rate | 0.833    |
| time/               |          |
|    episodes         | 176      |
|    fps              | 623      |
|    time_elapsed     | 2        |
|    total_timesteps  | 1760     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 181      |
|    exploration_rate | 0.829    |
| time/               |          |
|    episodes         | 180      |
|    fps              | 623      |
|    time_elapsed     | 2        |
|    total_timesteps  | 1800     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 186      |
|    exploration_rate | 0.825    |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 259      |
|    exploration_rate | 0.749    |
| time/               |          |
|    episodes         | 264      |
|    fps              | 603      |
|    time_elapsed     | 4        |
|    total_timesteps  | 2640     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 252      |
|    exploration_rate | 0.745    |
| time/               |          |
|    episodes         | 268      |
|    fps              | 605      |
|    time_elapsed     | 4        |
|    total_timesteps  | 2680     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 240      |
|    exploration_rate | 0.742    |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 164      |
|    exploration_rate | 0.666    |
| time/               |          |
|    episodes         | 352      |
|    fps              | 598      |
|    time_elapsed     | 5        |
|    total_timesteps  | 3520     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 183      |
|    exploration_rate | 0.662    |
| time/               |          |
|    episodes         | 356      |
|    fps              | 598      |
|    time_elapsed     | 5        |
|    total_timesteps  | 3560     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 181      |
|    exploration_rate | 0.658    |
| time/               |          |
|    episodes       

...

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 4.44e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 9684     |
|    fps              | 333      |
|    time_elapsed     | 290      |
|    total_timesteps  | 96840    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 151      |
|    n_updates        | 11709    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 4.44e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 9688     |
|    fps              | 333      |
|    time_elapsed     | 290      |
|    total_timesteps  | 96880    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 251      |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 4.42e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 9748     |
|    fps              | 331      |
|    time_elapsed     | 293      |
|    total_timesteps  | 97480    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 151      |
|    n_updates        | 11869    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 4.42e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 9752     |
|    fps              | 331      |
|    time_elapsed     | 294      |
|    total_timesteps  | 97520    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 251      |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 4.44e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 9812     |
|    fps              | 329      |
|    time_elapsed     | 297      |
|    total_timesteps  | 98120    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 101      |
|    n_updates        | 12029    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 4.46e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 9816     |
|    fps              | 329      |
|    time_elapsed     | 297      |
|    total_timesteps  | 98160    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 101      |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 4.49e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 9876     |
|    fps              | 328      |
|    time_elapsed     | 300      |
|    total_timesteps  | 98760    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 50.3     |
|    n_updates        | 12189    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 4.48e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 9880     |
|    fps              | 328      |
|    time_elapsed     | 300      |
|    total_timesteps  | 98800    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 151      |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 4.44e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 9940     |
|    fps              | 327      |
|    time_elapsed     | 303      |
|    total_timesteps  | 99400    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 151      |
|    n_updates        | 12349    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10       |
|    ep_rew_mean      | 4.44e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 9944     |
|    fps              | 327      |
|    time_elapsed     | 303      |
|    total_timesteps  | 99440    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 101      |
|    n_updates      

# Save model

In [10]:
model_path = os.path.join('saved_models', 'DQN_1')
model.save(model_path)

# Load model

In [11]:
model_path = os.path.join('saved_models', 'DQN_1')
env.reset()
model = DQN.load(model_path, env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


# Evaluate

In [12]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

C:\ProgramData\Anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(4708.040466308594, 0.0)

# Tensorboard    

In [ ]:
training_log_path = os.path.join('logs', 'DQN_3')
!tensorboard --logdir={training_log_path}